In [1]:
import os
import shutil
from pathlib import Path
import numpy as np
import random

np.random.seed(0) 
random.seed(0)


In [10]:
import yaml
import shutil
import json
import numpy as np
from pathlib import Path

try:
    import torch
    from ultralytics import YOLO
    torch_available = True
except ImportError:
    print("PyTorch/YOLO não disponível - apenas processamento de dataset")
    torch_available = False

try:
    import cv2
    cv2_available = True
except ImportError:
    cv2_available = False

try:
    from PIL import Image
    pil_available = True
except ImportError:
    print("PIL não disponível")
    pil_available = False

try:
    import mediapipe as mp
    mediapipe_available = True
except ImportError:
    print("MediaPipe não disponível")
    mediapipe_available = False

class SurgicalHandsIntegrator:
    def __init__(self, hands_dataset_path):
        """
        Integrador para dataset de mãos cirúrgicas com anotações JSON
        
        Args:
            hands_dataset_path: Caminho para o dataset surgical_hands_realise
        """
        self.hands_dataset_path = Path(hands_dataset_path)
        self.images_dir = self.hands_dataset_path / "images"
        self.annotations_file = self.hands_dataset_path / "annotations.json"
        
        # Backup: MediaPipe para casos sem anotações
        if mediapipe_available:
            self.mp_hands = mp.solutions.hands
            self.hands = self.mp_hands.Hands(
                static_image_mode=True,
                max_num_hands=4,
                min_detection_confidence=0.5,
                min_tracking_confidence=0.5
            )
            self.mediapipe_available = True
        else:
            self.mediapipe_available = False
            print("MediaPipe não disponível - usando apenas anotações JSON")

    def find_all_images(self):
        """Encontra todas as imagens em todas as subpastas"""
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        all_images = []
        
        print(f"Procurando imagens em: {self.images_dir}")
        
        for subfolder in self.images_dir.iterdir():
            if subfolder.is_dir():
                print(f"Verificando pasta: {subfolder.name}")
                folder_images = []
                
                for ext in image_extensions:
                    folder_images.extend(list(subfolder.glob(ext)))
                    folder_images.extend(list(subfolder.glob(ext.upper())))
                
                print(f"  Encontradas {len(folder_images)} imagens")
                all_images.extend(folder_images)
        
        print(f"Total de imagens encontradas: {len(all_images)}")
        return all_images

    def load_hand_annotations(self):
        """Carrega e analisa anotações JSON"""
        try:
            print(f"Carregando anotações de: {self.annotations_file}")
            with open(self.annotations_file, 'r', encoding='utf-8') as f:
                annotations = json.load(f)
            
            print(f"✅ Anotações carregadas com sucesso")
            print(f"Tipo de dados: {type(annotations)}")
            
            # Analisar estrutura das anotações
            if isinstance(annotations, dict):
                print(f"Chaves disponíveis: {list(annotations.keys())}")
                if 'images' in annotations:
                    print(f"Número de imagens anotadas: {len(annotations['images'])}")
                if 'annotations' in annotations:
                    print(f"Número de anotações: {len(annotations['annotations'])}")
            elif isinstance(annotations, list):
                print(f"Lista com {len(annotations)} elementos")
                if annotations:
                    print(f"Exemplo de elemento: {list(annotations[0].keys()) if isinstance(annotations[0], dict) else type(annotations[0])}")
            
            return annotations
        except Exception as e:
            print(f"❌ Erro ao carregar anotações: {e}")
            return None

    def create_annotation_mapping(self, annotations):
        """Cria mapeamento eficiente das anotações JSON"""
        annotation_map = {}
        
        if isinstance(annotations, dict):
            if 'images' in annotations and 'annotations' in annotations:
                # Formato COCO
                print("📋 Detectado formato COCO")
                image_map = {img['id']: img['file_name'] for img in annotations['images']}
                
                for ann in annotations['annotations']:
                    image_id = ann['image_id']
                    if image_id in image_map:
                        filename = image_map[image_id]
                        if filename not in annotation_map:
                            annotation_map[filename] = []
                        annotation_map[filename].append(ann)
            else:
                # Formato direto
                print("📋 Detectado formato direto")
                annotation_map = annotations
        elif isinstance(annotations, list):
            # Lista de anotações
            print("📋 Detectado formato de lista")
            for ann in annotations:
                if isinstance(ann, dict):
                    filename = ann.get('filename', ann.get('file_name', ann.get('image_name', '')))
                    if filename:
                        annotation_map[filename] = ann
        
        print(f"📋 Mapeamento criado para {len(annotation_map)} imagens")
        return annotation_map

    def convert_json_to_yolo(self, annotation, image_width, image_height):
        """
        Converte anotação JSON para formato YOLO
        """
        yolo_annotations = []
        
        # Se a anotação for uma lista de objetos
        if isinstance(annotation, list):
            objects = annotation
        elif isinstance(annotation, dict):
            if 'annotations' in annotation:
                objects = annotation['annotations']
            elif 'bbox' in annotation or 'x' in annotation:
                objects = [annotation]
            else:
                objects = []
        else:
            objects = []
        
        for obj in objects:
            try:
                # Extrair informações da anotação
                if 'bbox' in obj:
                    bbox = obj['bbox']
                    if len(bbox) >= 4:
                        x, y, w, h = bbox[:4]
                    else:
                        continue
                elif 'x' in obj and 'y' in obj:
                    x, y = obj['x'], obj['y']
                    w, h = obj.get('width', obj.get('w', 50)), obj.get('height', obj.get('h', 50))
                else:
                    continue
                
                # Determinar classe da mão
                category = str(obj.get('category_name', obj.get('label', obj.get('class', obj.get('name', '')))))
                
                if any(word in category.lower() for word in ['left', 'esquerda', 'l_hand']):
                    class_id = 4  # left_hand
                elif any(word in category.lower() for word in ['right', 'direita', 'r_hand']):
                    class_id = 5  # right_hand
                elif 'hand' in category.lower() or 'mao' in category.lower():
                    # Se não especificado, usar heurística baseada na posição
                    class_id = 4 if (x + w/2) < image_width / 2 else 5
                else:
                    # Se não é claramente uma mão, pular
                    continue
                
                # Converter para formato YOLO (normalizado)
                x_center = (x + w / 2) / image_width
                y_center = (y + h / 2) / image_height
                width_norm = w / image_width
                height_norm = h / image_height
                
                # Validar coordenadas
                if 0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 < width_norm <= 1 and 0 < height_norm <= 1:
                    yolo_annotations.append([class_id, x_center, y_center, width_norm, height_norm])
                
            except Exception as e:
                print(f"⚠️ Erro ao processar anotação: {e}")
                continue
        
        return yolo_annotations

    def create_hand_detection_mediapipe(self, image_path):
        """Cria detecção de mão usando MediaPipe como fallback"""
        if not self.mediapipe_available or not pil_available:
            return []
        
        try:
            # Carregar imagem
            pil_image = Image.open(image_path).convert('RGB')
            image_array = np.array(pil_image)
            
            # Processar com MediaPipe
            results = self.hands.process(image_array)
            
            hand_detections = []
            if results.multi_hand_landmarks:
                h, w = image_array.shape[:2]
                
                for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    # Calcular bounding box
                    x_coords = [landmark.x * w for landmark in hand_landmarks.landmark]
                    y_coords = [landmark.y * h for landmark in hand_landmarks.landmark]
                    
                    x1, y1 = int(min(x_coords)), int(min(y_coords))
                    x2, y2 = int(max(x_coords)), int(max(y_coords))
                    
                    # Padding
                    padding = 30
                    x1 = max(0, x1 - padding)
                    y1 = max(0, y1 - padding)
                    x2 = min(w, x2 + padding)
                    y2 = min(h, y2 + padding)
                    
                    # Determinar lateralidade baseada na posição
                    center_x = (x1 + x2) / 2
                    class_id = 4 if center_x < w / 2 else 5  # left_hand : right_hand
                    
                    # Converter para formato YOLO
                    x_center = (x1 + x2) / 2 / w
                    y_center = (y1 + y2) / 2 / h
                    width = (x2 - x1) / w
                    height = (y2 - y1) / h
                    
                    hand_detections.append([class_id, x_center, y_center, width, height])
            
            return hand_detections
            
        except Exception as e:
            print(f"❌ Erro na detecção MediaPipe: {e}")
            return []

    def get_json_annotations_for_image(self, img_file, annotation_map):
        """Extrai anotações JSON para uma imagem específica"""
        img_filename = img_file.name
        img_stem = img_file.stem
        
        # Tentar diferentes formas de matching
        annotations_data = None
        
        # 1. Nome completo
        if img_filename in annotation_map:
            annotations_data = annotation_map[img_filename]
        # 2. Sem extensão
        elif img_stem in annotation_map:
            annotations_data = annotation_map[img_stem]
        # 3. Com diferentes extensões
        else:
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                test_name = f"{img_stem}{ext}"
                if test_name in annotation_map:
                    annotations_data = annotation_map[test_name]
                    break
        
        if not annotations_data:
            return []
        
        # Converter anotações JSON para formato YOLO
        try:
            if pil_available:
                pil_image = Image.open(img_file)
                w, h = pil_image.size
            else:
                # Fallback para estimativa
                w, h = 640, 480
            
            return self.convert_json_to_yolo(annotations_data, w, h)
        except Exception as e:
            print(f"❌ Erro ao converter anotações para {img_file.name}: {e}")
            return []

    def process_hands_dataset(self, output_dir):
        """
        Processa o dataset de mãos usando JSON primeiro, MediaPipe como fallback
        """
        output_dir = Path(output_dir)
        (output_dir / "images").mkdir(parents=True, exist_ok=True)
        (output_dir / "labels").mkdir(parents=True, exist_ok=True)
        
        # Encontrar todas as imagens
        all_images = self.find_all_images()
        
        if not all_images:
            print("❌ Nenhuma imagem encontrada!")
            return 0
        
        # Carregar anotações JSON
        annotations = self.load_hand_annotations()
        
        if not annotations:
            print("⚠️ Não foi possível carregar annotation.json - usando apenas MediaPipe")
            annotation_map = {}
        else:
            annotation_map = self.create_annotation_mapping(annotations)
        
        processed_count = 0
        json_count = 0
        mediapipe_count = 0
        
        for img_file in all_images:
            try:
                # PRIMEIRO: Tentar usar anotações JSON
                hand_detections = self.get_json_annotations_for_image(img_file, annotation_map)
                
                if hand_detections:
                    json_count += 1
                    print(f"✅ JSON: {img_file.name} ({len(hand_detections)} mãos)")
                else:
                    # FALLBACK: Usar MediaPipe se não houver anotações JSON
                    hand_detections = self.create_hand_detection_mediapipe(img_file)
                    if hand_detections:
                        mediapipe_count += 1
                        print(f"🤖 MediaPipe: {img_file.name} ({len(hand_detections)} mãos)")
                
                if hand_detections:
                    # Copiar/converter imagem
                    new_img_name = f"hand_{processed_count:06d}.jpg"
                    
                    if pil_available:
                        if img_file.suffix.lower() != '.jpg':
                            pil_image = Image.open(img_file).convert('RGB')
                            pil_image.save(output_dir / "images" / new_img_name, 'JPEG')
                        else:
                            shutil.copy(img_file, output_dir / "images" / new_img_name)
                    else:
                        shutil.copy(img_file, output_dir / "images" / new_img_name)
                    
                    # Salvar labels
                    label_file = output_dir / "labels" / f"hand_{processed_count:06d}.txt"
                    with open(label_file, 'w') as f:
                        for detection in hand_detections:
                            line = ' '.join(map(str, detection))
                            f.write(line + '\n')
                    
                    processed_count += 1
                    
                    if processed_count % 50 == 0:
                        print(f"📊 Processadas {processed_count} imagens (JSON: {json_count}, MediaPipe: {mediapipe_count})")
            
            except Exception as e:
                print(f"❌ Erro ao processar {img_file.name}: {e}")
                continue
        
        print(f"\n🎉 Dataset de mãos processado:")
        print(f"   Total: {processed_count} imagens")
        print(f"   JSON: {json_count} imagens")
        print(f"   MediaPipe: {mediapipe_count} imagens")
        
        return processed_count

    def __del__(self):
        """Limpar recursos"""
        if hasattr(self, 'hands') and self.mediapipe_available:
            self.hands.close()

class AdvancedYOLOTrainer:
    def __init__(self, base_model):
        self.base_model = base_model
        if torch_available:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = "cpu"
        
    def prepare_combined_dataset(self, surgical_data_path, hands_dataset_path):
        """
        Prepara dataset combinado com instrumentos cirúrgicos e mãos
        """
        print("=== Preparando Dataset Combinado ===")
        
        # Diretório de saída
        combined_dir = Path("combined_surgical_dataset")
        (combined_dir / "images/train").mkdir(parents=True, exist_ok=True)
        (combined_dir / "labels/train").mkdir(parents=True, exist_ok=True)
        
        # 1. Processar dataset original de instrumentos
        print("1. Processando dataset de instrumentos...")
        surgical_path = Path(surgical_data_path)
        
        instrument_count = 0
        if (surgical_path / "images/train").exists():
            for img_file in (surgical_path / "images/train").glob("*.jpg"):
                # Copiar imagem
                shutil.copy(img_file, combined_dir / "images/train" / img_file.name)
                
                # Copiar/criar label
                label_file = surgical_path / "labels/train" / f"{img_file.stem}.txt"
                target_label = combined_dir / "labels/train" / f"{img_file.stem}.txt"
                
                if label_file.exists():
                    shutil.copy(label_file, target_label)
                else:
                    target_label.touch()
                
                instrument_count += 1
        
        print(f"✅ Instrumentos processados: {instrument_count}")
        
        # 2. Processar dataset de mãos
        print("2. Processando dataset de mãos...")
        hands_integrator = SurgicalHandsIntegrator(hands_dataset_path)
        
        # Processar mãos para diretório temporário
        temp_hands_dir = Path("temp_hands_processed")
        hands_count = hands_integrator.process_hands_dataset(temp_hands_dir)
        
        # Mover imagens e labels processados para dataset combinado
        if temp_hands_dir.exists():
            for img_file in (temp_hands_dir / "images").glob("*.jpg"):
                shutil.move(img_file, combined_dir / "images/train" / img_file.name)
            
            for label_file in (temp_hands_dir / "labels").glob("*.txt"):
                shutil.move(label_file, combined_dir / "labels/train" / label_file.name)
            
            # Limpar diretório temporário
            shutil.rmtree(temp_hands_dir)
        
        print(f"✅ Mãos processadas: {hands_count}")
        
        # 3. Criar configuração do dataset
        total_images = instrument_count + hands_count
        
        data_config = {
            'path': str(combined_dir.absolute()),
            'train': 'images/train',
            'val': 'images/train',  # Usar mesmo conjunto para validação
            'nc': 6,
            'names': ['needle', 'clamp', 'curved_scissor', 'straight_scissor', 'left_hand', 'right_hand']
        }
        
        with open(combined_dir / "data.yaml", "w") as f:
            yaml.dump(data_config, f)
        
        print(f"\n=== Dataset Combinado Criado ===")
        print(f"📊 Total de imagens: {total_images}")
        print(f"🔧 Instrumentos: {instrument_count}")
        print(f"👋 Mãos: {hands_count}")
        print(f"🏷️ Classes: {data_config['names']}")
        
        return combined_dir

    def create_advanced_config(self):
        """Configuração avançada para treino YOLO"""
        config = {
            'lr0': 0.001,
            'lrf': 0.01,
            'momentum': 0.98,
            'weight_decay': 0.0005,
            'warmup_epochs': 10,
            'warmup_momentum': 0.8,
            'warmup_bias_lr': 0.1,
            'box': 7.5,
            'cls': 0.5,
            'dfl': 1.5,
            'epochs': 150,
            'batch': 8,
            'imgsz': 640,
            'patience': 50,
            'save_period': 10,
            'hsv_h': 0.015,
            'hsv_s': 0.7,
            'hsv_v': 0.4,
            'degrees': 15.0,
            'translate': 0.1,
            'scale': 0.5,
            'shear': 5.0,
            'perspective': 0.0001,
            'flipud': 0.0,
            'fliplr': 0.5,
            'mosaic': 1.0,
            'mixup': 0.15,
            'copy_paste': 0.3,
            'dropout': 0.2,
            'val': True,
            'plots': True,
            'verbose': True
        }
        return config

    def train_with_advanced_features(self, data_path, use_optimization=True):
        """Treina o modelo com configurações avançadas"""
        if not torch_available:
            print("❌ PyTorch não disponível - não é possível treinar")
            return None, 0.0
        
        config = self.create_advanced_config()
        
        print("🚀 Iniciando treino com dataset combinado...")
        model = YOLO(self.base_model)
        
        results = model.train(
            data=data_path,
            **config
        )
        
        best_map = results.results_dict.get('metrics/mAP50(B)', 0.0)
        best_model_path = 'runs/detect/train/weights/best.pt'
        
        return best_model_path, best_map

class SurgicalMultiTracker:
    """Classe para tracking de instrumentos e mãos"""
    def __init__(self, model_path):
        if torch_available:
            self.model = YOLO(model_path)
        else:
            print("❌ PyTorch não disponível - tracking não disponível")
            self.model = None
    
    def track_video_with_hands(self, input_video, output_video):
        """Método placeholder para tracking de vídeo"""
        if self.model is None:
            print("❌ Modelo não carregado")
            return
        
        print(f"🎥 Tracking de {input_video} para {output_video}")
        # Implementação do tracking aqui

In [11]:
def main():
    """Função principal"""
    
    # Configurações de caminhos
    surgical_dataset_path = Path("yolo_dataset")
    hands_dataset_path = Path("surgical_hands_release")
    
    # Verificar se os datasets existem
    if not surgical_dataset_path.exists():
        print(f"❌ Dataset de instrumentos não encontrado: {surgical_dataset_path}")
        return
    
    if not hands_dataset_path.exists():
        print(f"❌ Dataset de mãos não encontrado: {hands_dataset_path}")
        return
    
    # Preparar dataset original se necessário
    print("📁 Preparando dataset original de instrumentos...")
    yolo_dir = Path("yolo_dataset")
    if not yolo_dir.exists():
        (yolo_dir / "images/train").mkdir(parents=True, exist_ok=True)
        (yolo_dir / "labels/train").mkdir(parents=True, exist_ok=True)
        
        # Copiar dados originais
        img_dir = surgical_dataset_path / "Images"
        label_dir = surgical_dataset_path / "Labels/label object names"
        
        if img_dir.exists() and label_dir.exists():
            for label_file in label_dir.glob("*.txt"):
                img_id = label_file.stem
                img_file = img_dir / f"{img_id}.jpg"
                if img_file.exists():
                    shutil.copy(img_file, yolo_dir / "images/train" / f"{img_id}.jpg")
                    shutil.copy(label_file, yolo_dir / "labels/train" / f"{img_id}.txt")
    
    # Inicializar trainer
    trainer = AdvancedYOLOTrainer("yolo11n.pt")
    
    # Preparar dataset combinado
    combined_dataset_path = trainer.prepare_combined_dataset(
        surgical_data_path=yolo_dir,
        hands_dataset_path=hands_dataset_path
    )
    
    # Treinar modelo se PyTorch estiver disponível
    if torch_available:
        data_config_path = combined_dataset_path / "data.yaml"
        best_model_path, best_map = trainer.train_with_advanced_features(
            data_path=data_config_path,
            use_optimization=False
        )
        
        print(f"\n=== TREINO CONCLUÍDO ===")
        print(f"🎯 Melhor mAP@0.5: {best_map:.4f}")
        print(f"💾 Modelo salvo em: {best_model_path}")
        
        # Inicializar sistema de tracking
        if best_model_path and Path(best_model_path).exists():
            print("\n=== SISTEMA DE TRACKING PRONTO ===")
            tracker = SurgicalMultiTracker(best_model_path)
            print("🏷️ Classes detectáveis:")
            print("  0: needle (agulha)")
            print("  1: clamp (pinça)")
            print("  2: curved_scissor (tesoura curva)")
            print("  3: straight_scissor (tesoura reta)")
            print("  4: left_hand (mão esquerda)")
            print("  5: right_hand (mão direita)")
            
            print("\n🚀 Para usar o tracking:")
            print("tracker.track_video_with_hands('video_input.mp4', 'video_output.mp4')")
    else:
        print("\n⚠️ PyTorch não disponível - apenas processamento de dataset concluído")
        print("📦 Dataset combinado criado em:", combined_dataset_path)

if __name__ == "__main__":
    main()


📁 Preparando dataset original de instrumentos...
=== Preparando Dataset Combinado ===
1. Processando dataset de instrumentos...
✅ Instrumentos processados: 3009
2. Processando dataset de mãos...
Procurando imagens em: surgical_hands_release/images
Verificando pasta: 0318dd3e7e3_000200_000208
  Encontradas 60 imagens
Verificando pasta: K4WxM7z1PKo_000154_000202
  Encontradas 60 imagens
Verificando pasta: alzo8uZzhpk_000041_000049
  Encontradas 60 imagens
Verificando pasta: 9DhftZeReiI_000733_001020_1
  Encontradas 150 imagens
Verificando pasta: K4WxM7z1PKo_000327_000335
  Encontradas 60 imagens
Verificando pasta: eb984ec0cf5_000100_000108
  Encontradas 60 imagens
Verificando pasta: 3745b0bac16_000840_000848
  Encontradas 60 imagens
Verificando pasta: B5K_QYc_Y2o_000503_000511
  Encontradas 60 imagens
Verificando pasta: GkfnPAgdaUI_000128_000136
  Encontradas 60 imagens
Verificando pasta: 5uKdXnreV1s_000021_000029
  Encontradas 60 imagens
Verificando pasta: ZjjgFgisJwI_000145_000420_1
  

W0000 00:00:1748364160.341326  550048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1748364160.368483  550048 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


🤖 MediaPipe: 001225.png (1 mãos)
🤖 MediaPipe: 001214.png (2 mãos)
🤖 MediaPipe: 001180.png (1 mãos)
🤖 MediaPipe: 001166.png (2 mãos)
🤖 MediaPipe: 001196.png (1 mãos)
🤖 MediaPipe: 001197.png (1 mãos)
🤖 MediaPipe: 001164.png (2 mãos)
🤖 MediaPipe: 001162.png (2 mãos)
🤖 MediaPipe: 001198.png (1 mãos)
🤖 MediaPipe: 001174.png (1 mãos)
🤖 MediaPipe: 001260.png (1 mãos)
🤖 MediaPipe: 001301.png (1 mãos)
🤖 MediaPipe: 001258.png (1 mãos)
🤖 MediaPipe: 001215.png (1 mãos)
🤖 MediaPipe: 001235.png (1 mãos)
🤖 MediaPipe: 001292.png (1 mãos)
🤖 MediaPipe: 001259.png (1 mãos)
🤖 MediaPipe: 001169.png (1 mãos)
🤖 MediaPipe: 001161.png (1 mãos)
🤖 MediaPipe: 001191.png (1 mãos)
🤖 MediaPipe: 001222.png (1 mãos)
🤖 MediaPipe: 001195.png (1 mãos)
🤖 MediaPipe: 001211.png (1 mãos)
🤖 MediaPipe: 001188.png (1 mãos)
🤖 MediaPipe: 001185.png (1 mãos)
🤖 MediaPipe: 001204.png (1 mãos)
🤖 MediaPipe: 001181.png (1 mãos)
🤖 MediaPipe: 001218.png (1 mãos)
🤖 MediaPipe: 001213.png (1 mãos)
🤖 MediaPipe: 001178.png (2 mãos)
🤖 MediaPip

train: Scanning /mounts/grupo1/combined_surgical_dataset/labels/train... 3984 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3984/3984 [00:04<00:00, 934.66it/s] 


train: New cache created: /mounts/grupo1/combined_surgical_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 507.2±158.3 MB/s, size: 59.4 KB)


...

    150/150      1.41G      0.474     0.3135     0.8872          9        640: 100%|██████████| 498/498 [00:48<00:00, 10.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 249/249 [00:19<00:00, 12.64it/s]

                   all       3984       5287      0.969      0.969       0.99      0.903



150 epochs completed in 2.856 hours.
Optimizer stripped from runs/detect/train112/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train112/weights/best.pt, 5.5MB

Validating runs/detect/train112/weights/best.pt...
Ultralytics 8.3.139 🚀 Python-3.10.16 torch-2.7.0+cu128 CUDA:0 (NVIDIA RTX A6000, 48548MiB)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 249/249 [00:18<00:00, 13.52it/s]


                   all       3984       5287      0.969      0.969       0.99      0.902
                needle       1038       1038      0.986      0.987      0.994      0.894
                 clamp       1056       1056      0.961      0.977      0.991      0.873
        curved_scissor        971        971      0.991      0.987      0.995      0.957
      straight_scissor       1142       1142      0.995      0.996      0.995      0.959
             left_hand        610        634      0.953      0.927      0.981      0.862
            right_hand        429        446      0.931      0.938      0.983      0.869
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train112

=== TREINO CONCLUÍDO ===
🎯 Melhor mAP@0.5: 0.9897
💾 Modelo salvo em: runs/detect/train/weights/best.pt

=== SISTEMA DE TRACKING PRONTO ===
🏷️ Classes detectáveis:
  0: needle (agulha)
  1: clamp (pinça)
  2: curved_scissor (tesoura curva)
  3: straight_sci

In [12]:
import shutil

shutil.copy('runs/detect/train/weights/best.pt', 'yoloTrackingWithHands.pt')

'yoloTrackingWithHands.pt'